# separte workflow for coregistration of anatomical image to mni template

In [ ]:
# import modules
from os.path import join as opj
import os
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.ants import Registration
from nipype.interfaces.fsl import BET, Info

In [ ]:
# set paths and define parameters
experiment_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/'
output_dir = 'derivatives/preprocessing/output_registration'
working_dir = 'derivatives/preprocessing/workingdir_registration'

# list of subjects
#subject_list = ['sub-01']
subject_list = ['sub-15', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22', 'sub-23', 'sub-24']

# Smoothing widths to apply
fwhm = 3

# TR of functional images (from dataset)
TR = 1.45

# location of template file for coregistration to mni template (full path needed, error otherwise)
template = Info.standard_image('/media/lmn/86A406A0A406933B2/mni_icbm152_nlin_asym_09c_3mm/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii.gz')

In [ ]:
templates = {'anat': 'derivatives/mindboggle/freesurfer_subjects/{subject_id}/mri/brain.mgz'}
                
# SelectFiles - to grab the data (alternativ to DataGrabber)
selectfiles = Node(SelectFiles(templates,
                                base_directory=experiment_dir),
                   name="selectfiles")

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]


# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', '')]

datasink.inputs.substitutions = substitutions

In [ ]:
# skullstrip the ants-mni-template, because structural freesurfer-input is also skullstripped!
bet_mask = Node(BET(frac=0.5,
                    robust=True,
                    in_file=template,
                    output_type='NIFTI_GZ'),
                name="bet_mask")    

# antsRegistration - coregister t1 on mni template
registrationtemp = Node(Registration(args='--float',
                            collapse_output_transforms=True,
                            initial_moving_transform_com=True, 
                            num_threads=4,   # specify number of cores to use
                            output_inverse_warped_image=True,
                            output_warped_image=True,
                            sigma_units=['vox']*3,
                            transforms=['Rigid', 'Affine', 'SyN'],
                            terminal_output='file',
                            winsorize_lower_quantile=0.005,
                            winsorize_upper_quantile=0.995,
                            convergence_threshold=[1e-06],
                            convergence_window_size=[10],
                            metric=['MI', 'MI', 'CC'],
                            metric_weight=[1.0]*3,
                            number_of_iterations=[[1000, 500, 250, 100],
                                                  [1000, 500, 250, 100],
                                                  [100, 70, 50, 20]],
                            radius_or_number_of_bins=[32, 32, 4],
                            sampling_percentage=[0.25, 0.25, 1],
                            sampling_strategy=['Regular',
                                               'Regular',
                                               'None'],
                            shrink_factors=[[8, 4, 2, 1]]*3,
                            smoothing_sigmas=[[3, 2, 1, 0]]*3,
                            transform_parameters=[(0.1,),
                                                  (0.1,),
                                                  (0.1, 3.0, 0.0)],
                            use_histogram_matching=True,
                            write_composite_transform=True),
               name='registrationtemp')

In [ ]:
# Create a preprocessing workflow
registration = Workflow(name='registration')
registration.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow - aparc / ribbon mask 
registration.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),    
                  (selectfiles, registrationtemp, [('anat', 'moving_image')]),
                  (bet_mask, registrationtemp, [('out_file', 'fixed_image')]),
                  (registrationtemp, datasink, [('warped_image', 'registrationtemp.@warped_image'),
                                                ('inverse_warped_image', 'registrationtemp.@inverse_warped_image'),
                                                ('inverse_composite_transform', 'registrationtemp.@inverse_composite_transform'),
                                                ('composite_transform', 'registrationtemp.@composite_transform')]),
                      ])

In [ ]:
#### visualize the pipeline ####

# Create a colored mvpaflow output graph
registration.write_graph(graph2use='colored',format='png', simple_form=True)

# Visualize the simple graph
from IPython.display import Image
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/preprocessing/workingdir_registration/registration/graph.png')

In [ ]:
#### run the workflow using multiple cores ####
registration.run('MultiProc', plugin_args={'n_procs':4})

In [ ]:
!tree /media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/preprocessing/output_registration/